# Predict Heavy Equipment Auction Price

## Case Study Goal
Predict the sale price of a particular piece of heavy equipment at auction based
on it's usage, equipment type, and configuration.  The data is sourced from auction
result postings and includes information on usage and equipment configurations.

## Evaluation
The evaluation of your model will be based on Root Mean Squared Log Error.
Which is computed as follows:

![Root Mean Squared Logarithmic Error](images/rmsle.png)

where *p<sub>i</sub>* are the predicted values (predicted auction sale prices) 
and *a<sub>i</sub>* are the actual values (the actual auction sale prices).

Note that this loss function is sensitive to the *ratio* of predicted values to
the actual values, a prediction of 200 for an actual value of 100 contributes
approximately the same amount to the loss as a prediction of 2000 for an actual
value of 1000.  To convince yourself of this, recall that a difference of
logarithms is equal to a single logarithm of a ratio, and rewrite each summand
as a single logarithm of a ratio.

This loss function is implemented in score_model.py.

## Data
The data for this case study are in `./data`. Although there are both training
and testing data sets, the testing data set should only be utilized to evaluate
your final model performance at the end of the day.  Think about it as your
hold out set.  Use cross-validation on the training data set to identify your
best model and then score your best model on the test data at the end of the day.
By using the same test data and the same evaluation metric (RMSLE) the relative
performance of different group's models on this case study can be assessed.

A data dictionary is included that explains the columns in the data.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [2]:
auction_train = pd.read_csv('data/Train.csv', low_memory=False)

In [3]:
def getNullCount(df:pd.DataFrame) -> None:
    """Prints metrics of null values from a dataframe"""
    columns = df.columns
    for col in columns:
        total_nan = sum(pd.isna(df[col]))
        total_all = df[col].size
        print(f"Column: {col}  Total:{total_all}  Missing:{total_nan}  {round(total_nan/total_all, 2) * 100}%")
#getNullCount(auction_train)

In [4]:
auction_train.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
auction_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null int64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null object
saledate                    401125 non-null object
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
fiModelSeries               56908 non-null object
fiModelDescriptor           71919 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state                       4

In [6]:
auction_train.describe()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter
count,4.011250e+05,401125.000000,4.011250e+05,401125.000000,401125.000000,380989.000000,401125.000000,1.427650e+05
mean,1.919713e+06,31099.712848,1.217903e+06,6889.702980,134.665810,6.556040,1899.156901,3.457955e+03
std,9.090215e+05,23036.898502,4.409920e+05,6221.777842,8.962237,16.976779,291.797469,2.759026e+04
min,1.139246e+06,4750.000000,0.000000e+00,28.000000,121.000000,0.000000,1000.000000,0.000000e+00
25%,1.418371e+06,14500.000000,1.088697e+06,3259.000000,132.000000,1.000000,1985.000000,0.000000e+00
50%,1.639422e+06,24000.000000,1.279490e+06,4604.000000,132.000000,2.000000,1995.000000,0.000000e+00
75%,2.242707e+06,40000.000000,1.468067e+06,8724.000000,136.000000,4.000000,2000.000000,3.025000e+03
max,6.333342e+06,142000.000000,2.486330e+06,37198.000000,172.000000,99.000000,2013.000000,2.483300e+06


In [7]:
auction_train['UsageBand']

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: object

In [8]:
auction_train[(auction_train['YearMade'] < 1200)]['ModelID'].value_counts()

9550     488
3170     403
11587    377
8724     353
5561     296
        ... 
28548      1
1929       1
8074       1
12172      1
16424      1
Name: ModelID, Length: 3379, dtype: int64

# Drop Some Columns

In [9]:
auction_train.drop(columns=['UsageBand','Blade_Extension', 'Blade_Width', 'Enclosure_Type',
                           'Engine_Horsepower', 'Pushblock', 'Scarifier', 'Tip_Control',
                           'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
                           'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
                           'Differential_Type', 'Steering_Controls'], inplace=True)

In [10]:
auction_train.drop(columns='SalesID', inplace=True)

In [11]:
auction_train.drop(columns='fiBaseModel', inplace=True)

In [12]:
auction_train.drop(columns='fiSecondaryDesc', inplace=True)

In [13]:
auction_train.drop(columns='fiModelSeries', inplace=True)

In [14]:
auction_train.drop(columns='fiModelDescriptor', inplace=True)

In [15]:
auction_train.drop(columns='auctioneerID', inplace=True)

In [16]:
auction_train.head()

,SalePrice,MachineID,ModelID,datasource,YearMade,MachineHoursCurrentMeter,saledate,fiModelDesc,ProductSize,fiProductClassDesc,...,Hydraulics,Ripper,Tire_Size,Coupler,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type
0,66000,999089,3157,121,2004,68.0,11/16/2006 0:00,521D,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,...,2 Valve,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
1,57000,117657,77,121,1996,4640.0,3/26/2004 0:00,950FII,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,...,2 Valve,NaN,23.5,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
2,10000,434808,7009,121,2001,2838.0,2/26/2004 0:00,226,NaN,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,...,Auxiliary,NaN,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
3,38500,1026470,332,121,2001,3486.0,5/19/2011 0:00,PC120-6E,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",...,2 Valve,NaN,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
4,11000,1057373,17311,121,2007,722.0,7/23/2009 0:00,S175,NaN,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,...,Auxiliary,NaN,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN


# Convert MachineHoursCurrent Meter "NaN" values to the average value

In [17]:
auction_train['MachineHoursCurrentMeter'].fillna(auction_train['MachineHoursCurrentMeter'].mean(), inplace=True)

In [18]:
auction_train['MachineHoursCurrentMeter'].describe()

count    4.011250e+05
mean     3.457955e+03
std      1.645985e+04
min      0.000000e+00
25%      2.318000e+03
50%      3.457955e+03
75%      3.457955e+03
max      2.483300e+06
Name: MachineHoursCurrentMeter, dtype: float64

# Function for one hot encoding

In [19]:
# One Hot Encode Categoricals
def set_ohe(df:pd.DataFrame, col_name:str) -> None:
    for val in auction_train[col_name].value_counts().index:
        df[f"{col_name}: {val}"] = df[col_name].map(lambda x: 1.0 if x==val else 0.0 )

# Clean Ripper Values

In [20]:
auction_train['Ripper'].value_counts()

None or Unspecified    83452
Yes                     7902
Multi Shank             7633
Single Shank            5150
Name: Ripper, dtype: int64

In [21]:
set_ohe(auction_train, "Ripper")

In [22]:
auction_train.drop(columns='Ripper', inplace=True)

In [23]:
auction_train.head()

,SalePrice,MachineID,ModelID,datasource,YearMade,MachineHoursCurrentMeter,saledate,fiModelDesc,ProductSize,fiProductClassDesc,...,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Ripper: None or Unspecified,Ripper: Yes,Ripper: Multi Shank,Ripper: Single Shank
0,66000,999089,3157,121,2004,68.0,11/16/2006 0:00,521D,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1,57000,117657,77,121,1996,4640.0,3/26/2004 0:00,950FII,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
2,10000,434808,7009,121,2001,2838.0,2/26/2004 0:00,226,NaN,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,38500,1026470,332,121,2001,3486.0,5/19/2011 0:00,PC120-6E,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
4,11000,1057373,17311,121,2007,722.0,7/23/2009 0:00,S175,NaN,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


# Clean ProductSize Values

In [24]:
auction_train['ProductSize'].value_counts()

Medium            62274
Large / Medium    49678
Small             26494
Mini              24840
Large             20975
Compact            6089
Name: ProductSize, dtype: int64

In [25]:
set_ohe(auction_train, "ProductSize")

In [26]:
auction_train.drop(columns='ProductSize', inplace=True)

In [27]:
auction_train.head()

,SalePrice,MachineID,ModelID,datasource,YearMade,MachineHoursCurrentMeter,saledate,fiModelDesc,fiProductClassDesc,state,...,Ripper: None or Unspecified,Ripper: Yes,Ripper: Multi Shank,Ripper: Single Shank,ProductSize: Medium,ProductSize: Large / Medium,ProductSize: Small,ProductSize: Mini,ProductSize: Large,ProductSize: Compact
0,66000,999089,3157,121,2004,68.0,11/16/2006 0:00,521D,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,57000,117657,77,121,1996,4640.0,3/26/2004 0:00,950FII,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,10000,434808,7009,121,2001,2838.0,2/26/2004 0:00,226,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,New York,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,38500,1026470,332,121,2001,3486.0,5/19/2011 0:00,PC120-6E,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Texas,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,11000,1057373,17311,121,2007,722.0,7/23/2009 0:00,S175,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,New York,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Clean YearMade Values

If year is 1000, make it the average year?

In [28]:
import numpy as np

In [29]:
auction_train['YearMade'].replace(1000, np.NaN, inplace=True)

In [30]:
auction_train['YearMade'].fillna(auction_train['YearMade'].mean(), inplace=True)

In [31]:
auction_train.head()

,SalePrice,MachineID,ModelID,datasource,YearMade,MachineHoursCurrentMeter,saledate,fiModelDesc,fiProductClassDesc,state,...,Ripper: None or Unspecified,Ripper: Yes,Ripper: Multi Shank,Ripper: Single Shank,ProductSize: Medium,ProductSize: Large / Medium,ProductSize: Small,ProductSize: Mini,ProductSize: Large,ProductSize: Compact
0,66000,999089,3157,121,2004.0,68.0,11/16/2006 0:00,521D,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,57000,117657,77,121,1996.0,4640.0,3/26/2004 0:00,950FII,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,10000,434808,7009,121,2001.0,2838.0,2/26/2004 0:00,226,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,New York,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,38500,1026470,332,121,2001.0,3486.0,5/19/2011 0:00,PC120-6E,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Texas,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,11000,1057373,17311,121,2007.0,722.0,7/23/2009 0:00,S175,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,New York,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Clean fiProductClassDesc

In [33]:
# Clean fiProductClassDesc
# Create "Vehicle Type" Feature from "fiProductClassDesc"
auction_train["Vehicle Type"] = auction_train["ProductGroupDesc"]
auction_train["Power Rating"] = auction_train["fiProductClassDesc"].apply(lambda x:(x.partition("-")[-1]))
auction_train.drop(columns='fiProductClassDesc', inplace=True)

In [34]:
#auction_train["Vehicle Type"].value_counts()

In [35]:
set_ohe(auction_train, "Vehicle Type")
auction_train.drop(columns='Vehicle Type', inplace=True)

In [36]:
auction_train.head()

,SalePrice,MachineID,ModelID,datasource,YearMade,MachineHoursCurrentMeter,saledate,fiModelDesc,state,ProductGroup,...,ProductSize: Mini,ProductSize: Large,ProductSize: Compact,Power Rating,Vehicle Type: Track Excavators,Vehicle Type: Track Type Tractors,Vehicle Type: Backhoe Loaders,Vehicle Type: Wheel Loader,Vehicle Type: Skid Steer Loaders,Vehicle Type: Motor Graders
0,66000,999089,3157,121,2004.0,68.0,11/16/2006 0:00,521D,Alabama,WL,...,0.0,0.0,0.0,110.0 to 120.0 Horsepower,0.0,0.0,0.0,1.0,0.0,0.0
1,57000,117657,77,121,1996.0,4640.0,3/26/2004 0:00,950FII,North Carolina,WL,...,0.0,0.0,0.0,150.0 to 175.0 Horsepower,0.0,0.0,0.0,1.0,0.0,0.0
2,10000,434808,7009,121,2001.0,2838.0,2/26/2004 0:00,226,New York,SSL,...,0.0,0.0,0.0,1351.0 to 1601.0 Lb Operating Capacity,0.0,0.0,0.0,0.0,1.0,0.0
3,38500,1026470,332,121,2001.0,3486.0,5/19/2011 0:00,PC120-6E,Texas,TEX,...,0.0,0.0,0.0,12.0 to 14.0 Metric Tons,1.0,0.0,0.0,0.0,0.0,0.0
4,11000,1057373,17311,121,2007.0,722.0,7/23/2009 0:00,S175,New York,SSL,...,0.0,0.0,0.0,1601.0 to 1751.0 Lb Operating Capacity,0.0,0.0,0.0,0.0,1.0,0.0


In [38]:
auction_train.drop(columns='ProductGroupDesc', inplace=True)

# Get year of sale only from saledate

## Convert saledate to datetime object

In [39]:
auction_train['saledate'] = pd.to_datetime(auction_train['saledate'])

In [40]:
print(auction_train['saledate'][0])
print(type(auction_train['saledate'][0]))

2006-11-16 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [41]:
auction_train['yearsold'] = auction_train['saledate'].map(lambda x: x.year)

In [42]:
auction_train.drop(columns='saledate', inplace=True)

In [43]:
auction_train.head()

,SalePrice,MachineID,ModelID,datasource,YearMade,MachineHoursCurrentMeter,fiModelDesc,state,ProductGroup,Drive_System,...,ProductSize: Large,ProductSize: Compact,Power Rating,Vehicle Type: Track Excavators,Vehicle Type: Track Type Tractors,Vehicle Type: Backhoe Loaders,Vehicle Type: Wheel Loader,Vehicle Type: Skid Steer Loaders,Vehicle Type: Motor Graders,yearsold
0,66000,999089,3157,121,2004.0,68.0,521D,Alabama,WL,NaN,...,0.0,0.0,110.0 to 120.0 Horsepower,0.0,0.0,0.0,1.0,0.0,0.0,2006
1,57000,117657,77,121,1996.0,4640.0,950FII,North Carolina,WL,NaN,...,0.0,0.0,150.0 to 175.0 Horsepower,0.0,0.0,0.0,1.0,0.0,0.0,2004
2,10000,434808,7009,121,2001.0,2838.0,226,New York,SSL,NaN,...,0.0,0.0,1351.0 to 1601.0 Lb Operating Capacity,0.0,0.0,0.0,0.0,1.0,0.0,2004
3,38500,1026470,332,121,2001.0,3486.0,PC120-6E,Texas,TEX,NaN,...,0.0,0.0,12.0 to 14.0 Metric Tons,1.0,0.0,0.0,0.0,0.0,0.0,2011
4,11000,1057373,17311,121,2007.0,722.0,S175,New York,SSL,NaN,...,0.0,0.0,1601.0 to 1751.0 Lb Operating Capacity,0.0,0.0,0.0,0.0,1.0,0.0,2009


# Convert state names to state numbers

In [44]:
import us
us_dict = us.states.mapping('name', 'fips')
us_dict["Washington DC"] = us_dict.pop("District of Columbia")
us_dict["Unspecified"] = '0'
auction_train["state"]=auction_train["state"].map(lambda x: int(us_dict[x.strip()]))

In [45]:
getNullCount(auction_train)

Column: SalePrice  Total:401125  Missing:0  0.0%
Column: MachineID  Total:401125  Missing:0  0.0%
Column: ModelID  Total:401125  Missing:0  0.0%
Column: datasource  Total:401125  Missing:0  0.0%
Column: YearMade  Total:401125  Missing:0  0.0%
Column: MachineHoursCurrentMeter  Total:401125  Missing:0  0.0%
Column: fiModelDesc  Total:401125  Missing:0  0.0%
Column: state  Total:401125  Missing:0  0.0%
Column: ProductGroup  Total:401125  Missing:0  0.0%
Column: Drive_System  Total:401125  Missing:296764  74.0%
Column: Enclosure  Total:401125  Missing:325  0.0%
Column: Forks  Total:401125  Missing:209048  52.0%
Column: Pad_Type  Total:401125  Missing:321991  80.0%
Column: Ride_Control  Total:401125  Missing:252519  63.0%
Column: Stick  Total:401125  Missing:321991  80.0%
Column: Transmission  Total:401125  Missing:217895  54.0%
Column: Turbocharged  Total:401125  Missing:321991  80.0%
Column: Hydraulics  Total:401125  Missing:80555  20.0%
Column: Tire_Size  Total:401125  Missing:306407  76

In [52]:
auction_train.drop(columns='Drive_System', inplace=True)

In [46]:
#auction_train["Power Rating"] = auction_train["fiProductClassDesc"].apply(lambda x: (x.partition("-")[-1]))

In [47]:
#columns=['']

# Make a horsepower column

In [48]:
def getReMax(val:str) -> np.float:
    """Returns maximum number in a string using regex"""
    search = re.findall('\d+', val) 
    nums = map(np.float, search) 
    return max(nums)
#x = auction_train["Power Rating"][0]
#getReMax(x)

In [49]:
auction_train_hp = auction_train[auction_train["Power Rating"].str.contains('horsepower', case=False)]
auction_train['HorsePower'] = auction_train_hp['Power Rating'].map(getReMax)
auction_train['HorsePower'].fillna(auction_train['HorsePower'].mean(), inplace=True)
#auction_train['HorsePower'].isnull().sum()

In [50]:
auction_train.head()

,SalePrice,MachineID,ModelID,datasource,YearMade,MachineHoursCurrentMeter,fiModelDesc,state,ProductGroup,Drive_System,...,ProductSize: Compact,Power Rating,Vehicle Type: Track Excavators,Vehicle Type: Track Type Tractors,Vehicle Type: Backhoe Loaders,Vehicle Type: Wheel Loader,Vehicle Type: Skid Steer Loaders,Vehicle Type: Motor Graders,yearsold,HorsePower
0,66000,999089,3157,121,2004.0,68.0,521D,1,WL,NaN,...,0.0,110.0 to 120.0 Horsepower,0.0,0.0,0.0,1.0,0.0,0.0,2006,120.000000
1,57000,117657,77,121,1996.0,4640.0,950FII,37,WL,NaN,...,0.0,150.0 to 175.0 Horsepower,0.0,0.0,0.0,1.0,0.0,0.0,2004,175.000000
2,10000,434808,7009,121,2001.0,2838.0,226,36,SSL,NaN,...,0.0,1351.0 to 1601.0 Lb Operating Capacity,0.0,0.0,0.0,0.0,1.0,0.0,2004,165.216455
3,38500,1026470,332,121,2001.0,3486.0,PC120-6E,48,TEX,NaN,...,0.0,12.0 to 14.0 Metric Tons,1.0,0.0,0.0,0.0,0.0,0.0,2011,165.216455
4,11000,1057373,17311,121,2007.0,722.0,S175,36,SSL,NaN,...,0.0,1601.0 to 1751.0 Lb Operating Capacity,0.0,0.0,0.0,0.0,1.0,0.0,2009,165.216455


In [53]:
auction_train.drop(columns='Power Rating', inplace=True)

# Drop more columns

In [56]:
auction_train.drop(columns=['ProductGroup', 'Enclosure'], inplace=True)

In [58]:
auction_train.drop(columns=['Pad_Type', 'fiModelDesc'], inplace=True)

In [60]:
auction_train.drop(columns=['Forks','Ride_Control','Stick'], inplace=True)

In [62]:
auction_train.drop(columns=['Transmission','Turbocharged','Hydraulics'], inplace=True)

In [64]:
auction_train.drop(columns=['Tire_Size','Coupler','Track_Type'], inplace=True)

In [67]:
auction_train.drop(columns=['Undercarriage_Pad_Width','Stick_Length','Thumb'], inplace=True)

In [69]:
auction_train.drop(columns=['Pattern_Changer','Grouser_Type'], inplace=True)

In [70]:
auction_train.head()

,SalePrice,MachineID,ModelID,datasource,YearMade,MachineHoursCurrentMeter,state,Ripper: None or Unspecified,Ripper: Yes,Ripper: Multi Shank,...,ProductSize: Large,ProductSize: Compact,Vehicle Type: Track Excavators,Vehicle Type: Track Type Tractors,Vehicle Type: Backhoe Loaders,Vehicle Type: Wheel Loader,Vehicle Type: Skid Steer Loaders,Vehicle Type: Motor Graders,yearsold,HorsePower
0,66000,999089,3157,121,2004.0,68.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2006,120.000000
1,57000,117657,77,121,1996.0,4640.0,37,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2004,175.000000
2,10000,434808,7009,121,2001.0,2838.0,36,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2004,165.216455
3,38500,1026470,332,121,2001.0,3486.0,48,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2011,165.216455
4,11000,1057373,17311,121,2007.0,722.0,36,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2009,165.216455


In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as scs
import statsmodels.api as sm
%matplotlib inline
pd.options.mode.chained_assignment = None

## Restrictions
Please use only *regression* methods for this case study.  The following techniques 
are legal

  - Linear Regression.
  - Logistic Regression.
  - Median Regression (linear regression by minimizing the sum of absolute deviations).
  - Any other [GLM](http://statsmodels.sourceforge.net/devel/glm.html).
  - Regularization: Ridge and LASSO.

You may use other models or algorithms as supplements (for example, in feature
engineering), but your final submissions must be scores from a linear type
model.

## Important Tips

1. This data is quite messy. Try to use your judgement about where your
cleaning efforts will yield the most results and focus there first.
2. Because of the restriction to linear models, you will have to carefully
consider how to transform continuous predictors in your model.
3. Remember any transformations you apply to the training data will also have
to be applied to the testing data, so plan accordingly.
4. Any transformations of the training data that *learn parameters* (for
example, standardization learns the mean and variance of a feature) must only
use parameters learned from the *training data*.
5. It's possible some columns in the test data will take on values not seen in
the training data. Plan accordingly.
6. Use your intuition to *think about where the strongest signal about a price
is likely to come from*. If you weren't fitting a model, but were asked to use
this data to predict a price what would you do? Can you combine the model with
your intuitive instincts?  This is important because it can be done *without
looking at the data*; thinking about the problem has no risk of overfitting.
7. Start simply. Fit a basic model and make sure you're able to get the
submission working then iterate to improve. 
8. Remember that you are evaluated on a loss function that is only sensitive to
the *ratios* of predicted to actual values.  It's almost certainly too much of
a task to implement an algorithm that minimizes this loss function directly in
the time you have, but there are some steps you can take to do a good job of
it.    

## Overview of the score_model.py script
Included is a score function to test your predictions of the test set against the provided hold out test set.  This follows a common setup in competitions such as Kaggle, where this came from.  In these types of setups, there is a labeled train set to do your modeling and feature tuning.  There is also a provided hold-out test set to compare your predictions against.  You will need to fit a model on the training data and get a prediction for all the data in the test set.  You will then need to create csv containing the field 'SalesID' and 'SalePrice' (must match exactly).  This will be the input parameter to running the function.    
Example:
In terminal:
```
python score_model.py <path to csv file>
```


## Credit
This case study is based on [Kaggle's Blue Book for Bulldozers](https://www.kaggle.com/c/bluebook-for-bulldozers) competition.  The best RMSLE was only 0.23 (obviously lower is better).  Note
that if you were to simply guess the median auction price for all the pieces of equipment in
the test set you would get an RMSLE of about 0.7.